In [1]:
# Run these following 3 installs to if you don't have shapely, geopands, etc
# if you are using sagemaker, you should use the "conda_python3" kernel
#!pip install laspy
#!pip install geopandas
#!pip install shapely

In [2]:
# Run these following 2 installs to install/upgrade database interface modules
#!pip install sqlalchemy psycopg2
#!pip install psycopg2-binary
!pip install geoalchemy2

You are using pip version 10.0.1, however version 20.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
### install/upgrade folium mapping tools
# !pip install folium

In [4]:
import boto3
import json

In [5]:
import numpy as np
import pandas as pd
import psycopg2
import sqlalchemy as db
import matplotlib as plt

%matplotlib inline

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [6]:
from sqlalchemy import update 
from geoalchemy2 import Geometry
from sqlalchemy.dialects import postgresql

In [7]:
#Get information from Secrets Manager, especially about the database
secretsmanager_client = boto3.client('secretsmanager')
dbSecretValues = secretsmanager_client.get_secret_value(SecretId='Postgres-GISDB')

In [8]:
dbSecretValuesJson = json.loads(dbSecretValues['SecretString'])
#uncomment the following line to confirm the Database access information
#dbSecretValuesJson 

In [9]:
from sqlalchemy import create_engine
# Postgres username, password, and database name
POSTGRES_ADDRESS = dbSecretValuesJson['POSTGRES_ADDRESS'] ## INSERT YOUR DB ADDRESS IF IT'S NOT ON PANOPLY
POSTGRES_PORT = int(dbSecretValuesJson['POSTGRES_PORT'])
POSTGRES_USERNAME = dbSecretValuesJson['POSTGRES_USERNAME'] ## CHANGE THIS TO YOUR PANOPLY/POSTGRES USERNAME
POSTGRES_PASSWORD = dbSecretValuesJson['POSTGRES_PASSWORD'] ## CHANGE THIS TO YOUR PANOPLY/POSTGRES PASSWORD POSTGRES_DBNAME = 'database' ## CHANGE THIS TO YOUR DATABASE NAME
POSTGRES_DBNAME = dbSecretValuesJson['POSTGRES_DBNAME']
# A long string that contains the necessary Postgres login information
# postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(
#     username=POSTGRES_USERNAME,
#     password=POSTGRES_PASSWORD,
#     ipaddress=POSTGRES_ADDRESS,
#     port=POSTGRES_PORT,
#     dbname=POSTGRES_DBNAME))
postgres_str = f"postgresql://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_ADDRESS}/{POSTGRES_DBNAME}"

engine = create_engine(postgres_str)
connection = engine.connect()
metadata = db.MetaData()
building_detailed_info_table = db.Table('building_detailed_info_within_ajjits_selected_tracts', metadata, autoload=True, autoload_with=engine)

In [21]:
#Get the las grid that belong to a building, get the percentil numbers and enter them into the table
#engie is the postgres sqlalchemy engine
def query_and_update_sql_table_with_height_pct(bldgid, engine, connection, building_detailed_info_table): 
#     sql_string = "select * from building_detailed_info_within_ajjits_selected_tracts where bldgid = 59026".format(bldgid)
#     pd.read_sql_query(sql_string, engine)
    sql_string = """select las_grnd_coord, lidar_z, intensity, classification, scan_angle, synthetic, withheld from lidarbuildingmatchtest 
        where bldgid = {0} and classification <> ALL (ARRAY[3, 4, 5])""".format(bldgid) 
    #the sql string removes all vegetation related points
    las_within_bldg_border_df = pd.read_sql_query(sql_string, engine)
    if(las_within_bldg_border_df.size > 0):
        las_qt_results_df = las_within_bldg_border_df[['lidar_z']].quantile([.999, .995, .99, .95, .90])
        pct999 = las_qt_results_df.loc[.999, 'lidar_z']
        pct995 = las_qt_results_df.loc[.995, 'lidar_z']
        pct990 = las_qt_results_df.loc[.990, 'lidar_z']
        pct950 = las_qt_results_df.loc[.950, 'lidar_z']
        pct900 = las_qt_results_df.loc[.900, 'lidar_z']

        query = db.update(building_detailed_info_table).values({'pct999': pct999, 'pct995': pct995, 'pct990': pct990, 'pct950': pct950})
        query = query.where(building_detailed_info_table.columns.bldgid == bldgid)
        ResultProxy = connection.execute(query)
        ResultProxy

In [22]:
sql_string = """select las_grnd_coord, lidar_z, intensity, classification, scan_angle, synthetic, withheld from lidarbuildingmatchtest 
        where bldgid = {0}""".format(70414)
las_within_bldg_border_df = pd.read_sql_query(sql_string, engine)
las_within_bldg_border_df

,las_grnd_coord,lidar_z,intensity,classification,scan_angle,synthetic,withheld
0,01010000807B721230FF4253C0DEA449031673434085EB...,23.02,9916.0,4,-3.0,0,0
1,01010000808838C929FF4253C0747B8902167343407B14...,21.48,18614.0,4,-3.0,0,0


In [23]:
query_and_update_sql_table_with_height_pct(59945, engine, connection, building_detailed_info_table)

In [24]:
sql_string = """select bldgid from building_detailed_info_within_ajjits_selected_tracts"""
bldgid_within_ajjit_tracts = pd.read_sql_query(sql_string, engine)
bldgid_within_ajjit_tracts.head()

,bldgid
0,59026
1,59450
2,59547
3,59576
4,59727


In [25]:
for index, row in bldgid_within_ajjit_tracts.iterrows():
    print (index, row['bldgid'])
    bldgid = np.int(row['bldgid'])
    query_and_update_sql_table_with_height_pct(bldgid, engine, connection, building_detailed_info_table)

0 59026
1 59450
2 59547
3 59576
4 59727
5 59730
6 59769
7 59889
8 59930
9 59909
10 60188
11 60220
12 60285
13 60289
14 60416
15 60421
16 60512
17 60517
18 60526
19 60527
20 60528
21 60558
22 60634
23 60641
24 60660
25 60769
26 60771
27 60806
28 60855
29 61016
30 61069
31 61095
32 61556
33 61584
34 61715
35 61747
36 61982
37 62036
38 62042
39 62135
40 62173
41 62174
42 62232
43 62283
44 62324
45 62378
46 62457
47 62538
48 62542
49 62630
50 62752
51 62736
52 62737
53 62749
54 62754
55 62824
56 62827
57 62845
58 62855
59 62857
60 62963
61 63035
62 63006
63 63011
64 63012
65 63015
66 63043
67 63072
68 63111
69 63128
70 63149
71 63169
72 63176
73 63193
74 63216
75 63273
76 63277
77 63358
78 63371
79 63428
80 63457
81 63467
82 63468
83 63478
84 63494
85 63553
86 63528
87 63563
88 63573
89 63578
90 63579
91 63591
92 63606
93 63630
94 63632
95 63640
96 63755
97 63812
98 63824
99 63849
100 63852
101 63853
102 63865
103 63875
104 63894
105 63898
106 63899
107 63900
108 63909
109 63935
110 63957


In [26]:
connection.close()
#terminate connections and drop the db engine
engine.dispose()